In [1]:
import numpy as np
import math

In [2]:
bits = 12
bytebits = 8
decoded_types = [np.uint8, np.uint16, np.uint32, np.uint64]
decoded_type = None
for t in decoded_types:
    if np.dtype(t).itemsize*bytebits >= bits:
        decoded_type = t
        break;
decoded_size = np.dtype(decoded_type).itemsize*bytebits

if decoded_size <= 32:
    encoded_type = np.uint32
elif decoded_size <= 64:
    encoded_type = np.uint64
else:
    raise Exception(decoded_size + " is too many bits")

encoded_size = np.dtype(encoded_type).itemsize*bytebits

gcd = math.gcd(bits, encoded_size)

# how many words of encoded_type an encoded block
encoded_stride = bits // gcd
# how many words of "bits" bits in an encoded block
decoded_stride = encoded_size // gcd

print(encoded_stride, decoded_stride)

3 8


In [3]:
total_bits = encoded_size * encoded_stride
encoded_bytes = np.dtype(encoded_type).itemsize

# Calculate the significance of a bit in a little-endian word, counting from the left
def shift_little(bitindex, bytesize, wordsize):
    byte = bitindex // bytesize
    rest = bitindex % bytesize
    return byte*bytesize - rest + bytesize - 1

# Calculate the significance of a bit in a big-endian word, counting from the left    
def shift_big(bitindex, bytesize, wordsize):
    return wordsize * bytesize - bitindex - 1

# Calculate mask and shift operations to encode and decode "bits" bits 
# packed little endian uints on a little endian machine
def little_on_little_op(b, bits, bytesize, encoded_bytes):
    encoded_size = bytesize * encoded_bytes
    decoded_index = shift_little(b, bytesize, encoded_bytes) // bits
    encoded_index = b // encoded_size
    decoded_shift = shift_little(b, bytesize, encoded_bytes) % bits    
    encoded_shift = shift_little(b % encoded_size, bytesize, encoded_bytes)
    return (decoded_index, encoded_index, decoded_shift, encoded_shift)

# Calculate mask and shift operations to encode and decode "bits" bits 
# packed big endian uints on a big endian machine
def big_on_big_op(b, bits, bytesize, encoded_bytes):
    decoded_index = b // bits
    encoded_index = b // encoded_size
    decoded_shift = shift_big(b % bits, bits, 1)
    encoded_shift = shift_big(b % encoded_size, bytesize, encoded_bytes)
    return (decoded_index, encoded_index, decoded_shift, encoded_shift)

def big_on_little_op(b, bits, bytesize, encoded_bytes):
    decoded_index = b // bits
    encoded_index = b // encoded_size
    decoded_shift = shift_big(b % bits, bits, 1)
    encoded_shift = shift_little(b % encoded_size, bytesize, encoded_bytes)
    return (decoded_index, encoded_index, decoded_shift, encoded_shift)

def little_on_big_op(b, bits, bytesize, encoded_bytes):
    decoded_index = shift_little(b, bytesize, encoded_bytes) // bits
    encoded_index = b // encoded_size
    decoded_shift = shift_little(b, bytesize, encoded_bytes) % bits
    encoded_shift = shift_big(b % encoded_size, bytesize, encoded_bytes)
    return (decoded_index, encoded_index, decoded_shift, encoded_shift)

def ops(total_bits, bits, bytesize, encoded_bytes, opfunc):
    ops = dict()
    for b in range(total_bits):
        (decoded_index, encoded_index, decoded_shift, encoded_shift) = opfunc(b, bits, bytesize, encoded_bytes)
        mask = 1 << decoded_shift
        net_shift = encoded_shift - decoded_shift
        # Combine processing operations that have same 
        # decoded and encoded index and same net shift
        # by OR-ing the bits together into one mask
        key = (decoded_index, encoded_index, net_shift)
        if key in ops:
            ops[key] |= mask
        else:
            ops[key] = mask
    return ops

def pretty_print(ops):
    for key in ops:
        mask = ops[key]
        print(key, hex(mask))

for f in (little_on_little_op, big_on_big_op, big_on_little_op, little_on_big_op):
    print()
    print(f.__name__)
    pretty_print(ops(total_bits, bits, bytebits, encoded_bytes, f))



little_on_little_op
(0, 0, 0) 0xfff
(1, 0, 12) 0xfff
(2, 0, 24) 0xff
(3, 1, 4) 0xfff
(2, 1, -8) 0xf00
(4, 1, 16) 0xfff
(5, 1, 28) 0xf
(5, 2, -4) 0xff0
(6, 2, 8) 0xfff
(7, 2, 20) 0xfff

big_on_big_op
(0, 0, 20) 0xfff
(1, 0, 8) 0xfff
(2, 0, -4) 0xff0
(2, 1, 28) 0xf
(3, 1, 16) 0xfff
(4, 1, 4) 0xfff
(5, 1, -8) 0xf00
(5, 2, 24) 0xff
(6, 2, 12) 0xfff
(7, 2, 0) 0xfff

big_on_little_op
(0, 0, -4) 0xff0
(0, 0, 12) 0xf
(1, 0, 0) 0xf00
(1, 0, 16) 0xff
(2, 0, 20) 0xff0
(2, 1, 4) 0xf
(3, 1, -8) 0xf00
(3, 1, 8) 0xff
(4, 1, 12) 0xff0
(4, 1, 28) 0xf
(5, 1, 16) 0xf00
(5, 2, 0) 0xff
(6, 2, 4) 0xff0
(6, 2, 20) 0xf
(7, 2, 8) 0xf00
(7, 2, 24) 0xff

little_on_big_op
(0, 0, 24) 0xff
(1, 0, 20) 0xf
(0, 0, 8) 0xf00
(1, 0, 4) 0xff0
(2, 0, 0) 0xff
(3, 1, 28) 0xf
(2, 1, 16) 0xf00
(3, 1, 12) 0xff0
(4, 1, 8) 0xff
(5, 1, 4) 0xf
(4, 1, -8) 0xf00
(5, 2, 20) 0xff0
(6, 2, 16) 0xff
(7, 2, 12) 0xf
(6, 2, 0) 0xf00
(7, 2, -4) 0xff0


In [23]:
def validate(bits, pattern, decoded_dtype, encoded_dtype, opfunc):
    bytebits = 8
    decoded_size = decoded_dtype.itemsize*bytebits
    encoded_size = encoded_dtype.itemsize*bytebits
    gcd = math.gcd(bits, encoded_size)

    encoded_stride = bits // gcd
    decoded_stride = encoded_size // gcd
    
    decoded = np.zeros(decoded_stride, dtype=decoded_dtype)
    decoded_result = np.zeros(decoded_stride, dtype=decoded_dtype)
    
    for i in range(decoded_stride):
        decoded[i] = pattern[i % len(pattern)]
    
    print(list(map(hex, decoded.view(np.uint8))))

    encoded = np.zeros(encoded_stride, dtype=encoded_dtype)
    total_bits = encoded_size * encoded_stride
    encoded_bytes = encoded_dtype.itemsize
    
    my_ops = ops(total_bits, bits, bytebits, encoded_bytes, opfunc)
    
    apply_encode_ops(decoded, encoded, my_ops)
    
    print(list(map(hex, encoded.view(np.uint8))))

    apply_decode_ops(encoded, decoded_result, my_ops)

    print(list(map(hex, decoded_result.view(np.uint8))))

def shift(i, shift):
    return (i << shift) if (shift > 0) else (i >> -shift)

def apply_encode_ops(decoded_block, encoded_block, ops):
    encoded_block.fill(0)
    encoded_dtype = encoded_block.dtype
    decoded_dtype = decoded_block.dtype
    for key in ops:
        mask = encoded_dtype.type(ops[key])
        (decoded_index, encoded_index, net_shift) = key
        encoded_block[encoded_index] |= encoded_dtype.type(shift((decoded_block[decoded_index] & mask), net_shift))
        
def apply_decode_ops(encoded_block, decoded_block, ops):
    decoded_block.fill(0)
    encoded_dtype = encoded_block.dtype
    decoded_dtype = decoded_block.dtype
    for key in ops:
        mask = encoded_dtype.type(ops[key])
        (decoded_index, encoded_index, net_shift) = key
        decoded_block[decoded_index] |= decoded_dtype.type(shift(encoded_block[encoded_index], -net_shift) & mask)

tests = (
    (np.dtype('>u2'), np.dtype('>u4'), big_on_big_op),
    (np.dtype('<u2'), np.dtype('>u4'), little_on_big_op),
    (np.dtype('<u2'), np.dtype('<u4'), little_on_little_op),
    (np.dtype('>u2'), np.dtype('<u4'), big_on_little_op)
)

for (decoded_dtype, encoded_dtype, func) in tests:
    print(func.__name__)
    validate(12, [0xabc, 0xdef, 0xaaa, 0xbbb, 0xccc, 0xfed, 0xcba, 0x001], decoded_dtype, encoded_dtype, func)
    print()

big_on_big_op
['0xa', '0xbc', '0xd', '0xef', '0xa', '0xaa', '0xb', '0xbb', '0xc', '0xcc', '0xf', '0xed', '0xc', '0xba', '0x0', '0x1']
['0xab', '0xcd', '0xef', '0xaa', '0xab', '0xbb', '0xcc', '0xcf', '0xed', '0xcb', '0xa0', '0x1']
['0xa', '0xbc', '0xd', '0xef', '0xa', '0xaa', '0xb', '0xbb', '0xc', '0xcc', '0xf', '0xed', '0xc', '0xba', '0x0', '0x1']

little_on_big_op
['0xbc', '0xa', '0xef', '0xd', '0xaa', '0xa', '0xbb', '0xb', '0xcc', '0xc', '0xed', '0xf', '0xba', '0xc', '0x1', '0x0']
['0xbc', '0xfa', '0xde', '0xaa', '0xba', '0xbb', '0xcc', '0xdc', '0xfe', '0xba', '0x1c', '0x0']
['0xbc', '0xa', '0xef', '0xd', '0xaa', '0xa', '0xbb', '0xb', '0xcc', '0xc', '0xed', '0xf', '0xba', '0xc', '0x1', '0x0']

little_on_little_op
['0xbc', '0xa', '0xef', '0xd', '0xaa', '0xa', '0xbb', '0xb', '0xcc', '0xc', '0xed', '0xf', '0xba', '0xc', '0x1', '0x0']
['0xbc', '0xfa', '0xde', '0xaa', '0xba', '0xbb', '0xcc', '0xdc', '0xfe', '0xba', '0x1c', '0x0']
['0xbc', '0xa', '0xef', '0xd', '0xaa', '0xa', '0xbb', '0xb'

In [ ]:
mask = encoded_type(0xfff)

decoded = np.zeros(decoded_stride, dtype=decoded_type)
decoded_result = np.zeros(decoded_stride, dtype=decoded_type)

decoded.fill(1)
print(decoded)

encoded = np.zeros(encoded_stride, dtype=encoded_type)

encoded[0] = (   (decoded[0] & mask)
              | ((decoded[1] & mask) << 12) 
              | ((decoded[2] & mask) << 24))
             
encoded[1] = (  ((decoded[2] & mask) >>  8)
              | ((decoded[3] & mask) <<  4)
              | ((decoded[4] & mask) << 16)
              | ((decoded[5] & mask) << 28))

encoded[2] = (  ((decoded[5] & mask) >>  4) 
              | ((decoded[6] & mask) <<  8)
              | ((decoded[7] & mask) << 20))

print(list(map(bin, encoded.view(np.uint8))))

decoded_result[0]  =   encoded[0]        & mask
decoded_result[1]  =  (encoded[0] >> 12) & mask
decoded_result[2]  = ((encoded[0] >> 24) & mask) | ((encoded[1] << 8) & mask)

decoded_result[3]  =  (encoded[1] >>  4) & mask
decoded_result[4]  =  (encoded[1] >> 16) & mask
decoded_result[5]  = ((encoded[1] >> 28) & mask) | ((encoded[2] << 4) & mask)

decoded_result[6]  =  (encoded[2] >>  8) & mask
decoded_result[7]  =  (encoded[2] >> 20) & mask

print(decoded_result)